In [1]:
from pathlib import Path
import subprocess
import os
from glob import glob
import numpy as np
import pandas as pd
from datetime import datetime
import joblib
import warnings
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.ensemble import HistGradientBoostingClassifier

from sklearn.metrics import roc_auc_score
from tqdm.auto import tqdm
from sklearn.model_selection import TimeSeriesSplit, GroupKFold, StratifiedGroupKFold
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import KNNImputer
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.datasets import load_wine
import pickle
import seaborn as sns

In [33]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForCausalLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("nllb-200-3.3B/tokenize")

model = AutoModelForSeq2SeqLM.from_pretrained(
    "1.3B-Out-chunk512/checkpoint-14850", device_map="auto"
)

model.to(device)
model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-23): 24 x M2M100EncoderLayer(
          (self_attn): M2M100SdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_features=1024, bias=True)
   

In [34]:
import torch
print(torch.__version__)           # Should show 2.2.2+cu118
print(torch.cuda.is_available())   # Should return True

2.6.0+cu124
True


In [35]:
data_train = pd.read_json(path_or_buf="Data_MT/mt_train.jsonl", lines=True)
data_val = pd.read_json(path_or_buf="Data_MT/mt_dev.jsonl", lines=True)
data_test = pd.read_json(path_or_buf="Data_MT/mt_test.jsonl", lines=True)

In [36]:
data_train = data_train.drop('context', axis=1)
data_val = data_val.drop('context', axis=1)

In [37]:
data_train

,source,translation
0,我想咨询件事 我对别人有点敌意,ฉันอยากจะถามอะไรหน่อย ฉันรู้สึกไม่เป็นมิตรกับค...
1,我考虑细菌性或滴虫性阴道炎的可能性,ฉันกำลังคิดว่าน่าจะเกิดภาวะช่องคลอดอักเสบจากเช...
2,你好！腰痛发现有多长时间了？是腰中间疼痛还是两侧肌肉疼痛？,สวัสดี คุณมีอาการปวดหลังส่วนล่างมานานแค่ไหนแล้...
3,是不是平时勤刷牙，多刷干净点就可以减少牙结石的沉积呢？,ถ้าแปรงฟันบ่อยๆ และทำความสะอาดบ่อยๆ ก็จะสามารถ...
4,周围环境的物理化学因素也可以引起身体过敏,ปัจจัยทางกายภาพและเคมีในสภาพแวดล้อมโดยรอบ ก็สา...
...,...,...
18595,[自动回复]您好，现在比较忙，请您留言具体问题，稍后我会及时回复您的问题，请谅解。,[ตอบกลับอัตโนมัติ] สวัสดี ตอนนี้ฉันค่อนข้างยุ่...
18596,[自动回复]您好，现在比较忙，请您留言具体问题，稍后我会及时回复您的问题，请谅解。,[ตอบกลับอัตโนมัติ] สวัสดี ตอนนี้หมอค่อนข้างยุ่...
18597,[自动回复]您好，现在比较忙，请您留言具体问题，稍后我会及时回复您的问题，请谅解。,[ตอบกลับอัตโนมัติ] สวัสดี ตอนนี้หมอค่อนข้างยุ่...
18598,[自动回复]您好，现在比较忙，请您留言具体问题，稍后我会及时回复您的问题，请谅解。,[ตอบกลับอัตโนมัติ] สวัสดี ตอนนี้ฉันค่อนข้างยุ่...


In [38]:
from datasets import Dataset, DatasetDict
from transformers import (
    NllbTokenizer, 
    M2M100ForConditionalGeneration,
    Trainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType
)
from torch.utils.data import DataLoader
import numpy as np
from sklearn.model_selection import train_test_split
import logging
from IPython.display import display, HTML
import matplotlib.pyplot as plt

class Config:
    SRC_LANG = "zho_Hans"  # Simplified Chinese
    TGT_LANG = "tha_Thai"  # Thai
    
    # Training parameters
    OUTPUT_DIR = "./1.3B-Out-chunk512"
    NUM_EPOCHS = 11
    BATCH_SIZE = 2  # Adjust based on your GPU memory
    LEARNING_RATE = 1e-5
    MAX_LENGTH = 1140

config = Config()

In [39]:
data_val['translation']

0       ตาพร่ามัวและหมดสติเนื่องจากสมองบาดเจ็บ (ชายอาย...
1       เดือนที่แล้ว ประจำเดือนครั้งแรกมาตอนต้นเดือน ค...
2                  บางคนแพ้ ทำแล้วไม่สบาย อึดอัดและปวดบวม
3       ความเครียดเยอะ เบื่ออาหารและน้ำ จะทำให้ผอมลงแล...
4              ความรู้สึกตอนนี้คือเหมือนมีอาการประสาทหลอน
                              ...                        
2995    สวัสดี หินปูนเป็นรอยโรคที่หลงเหลืออยู่หลังจากก...
2996            พี่สาวของฉันบอกให้กิน เธอเป็นสูตินรีแพทย์
2997          วิธีการรักษาโรคประสาทอ่อน? (ชาย อายุ 31 ปี)
2998    หลังการสัมผัสเชื้อมา 3 เดือนสามารถยืนยันได้อย่...
2999    ข้อใดข้อหนึ่งก็สามารถตัดความเสี่ยงของการติดเชื...
Name: translation, Length: 3000, dtype: object

In [40]:
x_train, x_test, y_train, y_test = train_test_split(
    data_val['source'], data_val['translation'], 
    test_size=0.05,  # 20% for test set
    random_state=42,  # for reproducibility
    shuffle=True,
)

In [41]:
# result = pd.concat([data_train, pd.DataFrame({"source": x_train, "translation": y_train})], axis=0)
result = pd.concat([data_train, data_val], axis=0)


In [42]:
datasets = DatasetDict({
    'train': Dataset.from_dict({'source': result['source'], 'translation': result['translation']}),
    # 'validation': Dataset.from_dict({'source': x_test, 'translation': y_test}),
})

In [43]:
datasets

DatasetDict({
    train: Dataset({
        features: ['source', 'translation'],
        num_rows: 21600
    })
})

In [44]:
tokenizer.src_lang = config.SRC_LANG
tokenizer.tgt_lang = config.TGT_LANG

In [14]:
# Tokenize with context (Option 2 - Batched)
def tokenize_function(examples):
    inputs = tokenizer(
        examples["source"],
        truncation=True,
        max_length=1140,
        return_tensors="pt",
        padding="max_length", 
    )
    labels = tokenizer(
        examples["translation"],
        truncation=True,
        max_length=1140,
        return_tensors="pt",
        padding="max_length"
    )
    inputs["labels"] = labels["input_ids"]
    return inputs

# Apply to dataset
tokenized_datasets = datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=datasets["train"].column_names,
)

Map:   0%|          | 0/21600 [00:00<?, ? examples/s]

In [15]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 21600
    })
})

In [16]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding="max_length"
)

In [17]:
training_args = TrainingArguments(
    output_dir=config.OUTPUT_DIR,
    num_train_epochs=config.NUM_EPOCHS,
    per_device_train_batch_size=config.BATCH_SIZE,
    gradient_accumulation_steps=8,
    warmup_steps=1000,
    learning_rate=config.LEARNING_RATE,
    weight_decay=0.01,
    logging_dir=f"{config.OUTPUT_DIR}/logs",
    logging_steps=1000,
    save_strategy="epoch",
    save_total_limit=2,
    fp16=torch.cuda.is_available(),
    report_to="tensorboard",
    remove_unused_columns=False,                             # Saves memory; set False if using custom dataset fields
    dataloader_pin_memory=True,                             # Helpful on some systems
    gradient_checkpointing=True,
    dataloader_num_workers=2,                               # Speeds up data loading; adjust as needed
    optim="adamw_torch",
)

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    # eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

/tmp/ipykernel_29674/190091860.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [19]:
def print_trainable_parameters(model):
    trainable = 0
    total = 0
    for param in model.parameters():
        total += param.numel()
        if param.requires_grad:
            trainable += param.numel()
    print(f"Trainable parameters: {trainable:,}")
    print(f"Total parameters:     {total:,}")
    print(f"Trainable %:          {100 * trainable / total:.2f}%")

print_trainable_parameters(model)

Trainable parameters: 1,370,638,336
Total parameters:     1,370,638,336
Trainable %:          100.00%


In [20]:
trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
1000,8.744400
2000,1.640300
3000,0.049100
4000,0.038300
5000,0.033900
6000,0.031800
7000,0.029900
8000,0.028600
9000,0.027200
10000,0.026500


/home/tb1017/.local/lib/python3.10/site-packages/transformers/modeling_utils.py:3465: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
TOKENIZERS_PARALLELISM=(true | false)
huggingface/t

TrainOutput(global_step=14850, training_loss=0.7252770780312895, metrics={'train_runtime': 40649.3752, 'train_samples_per_second': 5.845, 'train_steps_per_second': 0.365, 'total_flos': 1.801164436144128e+18, 'train_loss': 0.7252770780312895, 'epoch': 11.0})

In [57]:
modelx = trainer.model.to("cuda")

In [14]:
modelx = model

In [53]:
import json
def translate_text(chinese_text):
    # Tokenize input
    inputs = tokenizer(
        chinese_text, 
        truncation=True, 
        max_length=1140,
        padding="max_length",
        return_tensors="pt"              # Ensure PyTorch tensors
    ).to("cuda")
    
    # Move to GPU if available
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}
        modelx.cuda()
        
    # Generate translation
    with torch.no_grad():
        generated_tokens = modelx.generate(
            **inputs,
            forced_bos_token_id=256175,
            temperature=0.3,  # Lower for more consistent translation
            top_p=0.8,
            num_beams=10,  # ← This parameter
            repetition_penalty=1.1,
            do_sample = True,
            pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            early_stopping=True
        )
    
    # Decode the translation
    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    return translation

In [50]:
data_test['source'][0]

'[自动回复]您好，有急事，请您留言具体问题，我会及时回复您的问题，请谅解。'

In [51]:
translate_text(data_test['source'][0])

'[ตอบกลับอัตโนมัติ] สวัสดี มีเรื่องเร่งด่วน โปรดฝากคําถามไว้ ฉันจะตอบคําถามของคุณโดยเร็วที่สุด ขออภัยในความไม่สะดวก'

In [52]:
import tqdm
predictions = []
for text in tqdm.tqdm(x_test):
    predictions.append(translate_text(text))

predictions

  7%|▋         | 10/150 [00:08<01:57,  1.19it/s]


KeyboardInterrupt: 

In [ ]:
ref = []
ref = y_test
ref

In [ ]:
syn = {
    '้ํา' : '้ำ',
    '่ํา' : '่ำ',
    '๊ํา' : '๊ำ',
    '๋ํา' : '๋ำ',
    'ํา' : 'ำ',
    'เเ' : 'แ'
}
mem = []
for data in predictions:
    s = data
    for k,v in syn.items(): s = s.replace(k,v)
    mem.append(s)
predictions = mem

In [ ]:
from pythainlp.tokenize import word_tokenize
import sacrebleu

# Tokenize predictions and references
tokenized_preds = [" ".join(word_tokenize(pred)) for pred in predictions]
tokenized_refs = [" ".join(word_tokenize(r)) for r in ref]

# Compute BLEU with no tokenization (since we already tokenized)
score = sacrebleu.corpus_bleu(tokenized_preds, [tokenized_refs], tokenize="none")
print(f"BLEU score: {score.score:.2f}")
#29.09

In [30]:
with open("summit_MT/nllb-200-1.3B(fine-tune-super_epoch)(val).txt", "w", encoding="utf-8") as f:
    for item in predictions:
        f.write(f"{item}\n")

In [38]:
print(torch.__version__)           # Should show 2.2.2+cu118
print(torch.cuda.is_available())   # Should return True

2.6.0+cu124
True


In [54]:
import tqdm
syn = {
    '้ํา' : '้ำ',
    '่ํา' : '่ำ',
    '๊ํา' : '๊ำ',
    '๋ํา' : '๋ำ',
    'ํา' : 'ำ',
    'เเ' : 'แ'
}
answer = []
for text in tqdm.tqdm(data_test['source']):
    answer.append(translate_text(text))

100%|██████████| 2000/2000 [33:13<00:00,  1.00it/s]


In [55]:
translate_text(data_test['source'][0])

'[ตอบกลับอัตโนมัติ] สวัสดี มีเรื่องเร่งด่วน โปรดฝากคําถามไว้ ฉันจะตอบคําถามของคุณโดยเร็วที่สุด ขออภัยในความไม่สะดวก'

In [56]:
mem = []
for data in answer:
    s = data
    for k,v in syn.items(): s = s.replace(k,v)
    mem.append(s)
answer = mem

In [57]:
with open("summit_MT/nllb-200-1.3B(fine-tune-final).txt", "w", encoding="utf-8") as f:
    for item in answer:
        f.write(f"{item}\n")